In [1]:
from pathlib import Path
from datetime import datetime as dt
import pandas as pd


In [2]:
path="/home/nlu/Desktop/NER/Arabic-NER-SVM-master/ANERCorp.xlsx"


In [3]:
xlsx = pd.ExcelFile(path)

In [4]:
df=pd.read_excel(xlsx, header=None)

In [9]:
df

,text,label
0,فرانكفورت,B-LOC
1,(د,O
2,ب,O
3,أ),O
4,أعلن,O
5,اتحاد,B-ORG
6,صناعة,I-ORG
7,السيارات,I-ORG
8,في,O
9,ألمانيا,B-LOC


In [10]:
df = df.drop(2, 1)

ValueError: labels [2] not contained in axis

In [11]:
df = df.rename(columns={0: 'text', 1: 'label'})

In [12]:
df

,text,label
0,فرانكفورت,B-LOC
1,(د,O
2,ب,O
3,أ),O
4,أعلن,O
5,اتحاد,B-ORG
6,صناعة,I-ORG
7,السيارات,I-ORG
8,في,O
9,ألمانيا,B-LOC


In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.2)
train_arr = []
test_arr = []
train_lbl = []
test_lbl = []

In [15]:
train_arr=train['text'].astype(str)
train_lbl=train['label'].astype(str)
test_arr=test['text'].astype(str)
test_lbl=test['label'].astype(str)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer

In [17]:
vectorizer = CountVectorizer()
vectorizer.fit(train_arr)
train_mat = vectorizer.transform(train_arr)

In [18]:
tfidf = TfidfTransformer()
tfidf.fit(train_mat)
train_tfmat = tfidf.transform(train_mat)


In [19]:
test_mat = vectorizer.transform(test_arr)
test_tfmat = tfidf.transform(test_mat)


In [20]:
del df
del test_arr
del train_arr

In [21]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB

In [22]:
train_tfmat

<120228x28223 sparse matrix of type '<class 'numpy.float64'>'
	with 107803 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.svm import LinearSVC

In [24]:
lsvm=LinearSVC()
lsvm.fit(train_tfmat,train_lbl)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [26]:
y_pred_lsvm=lsvm.predict(test_tfmat)

In [27]:
test=['ألمانيا']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]

'B-LOC'

In [33]:
from sklearn.metrics import  accuracy_score
from sklearn import metrics

In [34]:
print("accuracy:", metrics.accuracy_score(test_lbl, y_pred_lsvm))

accuracy: 0.934426774902


In [35]:
phrase="شاهد أحمد مباراة فرنسا"
arr=phrase.split()

In [36]:
arr

['شاهد', 'أحمد', 'مباراة', 'فرنسا']

In [37]:
import sys
y=[]
token=[]
for x in arr:
    x=[x]
    test_str = vectorizer.transform(x)
    test_tfstr = tfidf.transform(test_str)
    test_tfstr.shape
    token.append(x)
    y.append(lsvm.predict(test_tfstr.toarray())[0])

In [38]:
df=pd.DataFrame(list(zip(token,y)),columns=['token','entity_type'])

In [39]:
df

,token,entity_type
0,[شاهد],O
1,[أحمد],B-PERS
2,[مباراة],O
3,[فرنسا],B-LOC
